In [1]:
import numpy as np
N=1000
means = np.random.normal(size=N)
stddevs = 1.+np.arange(N)%2  # half have sigma = 1, half have sigma = 2
variances = stddevs*stddevs
vals = np.random.normal(means,stddevs,size=N) + np.arange(0,N,1)//(N/10) - 4. # 10 mixture components with means -4..5

# additive Gaussian scan statistic: F(S_1..S_T) = (\sum_j (C_j^2 / 2B_j)) - (C_all^2 / 2B_all)
C = (vals-means)/variances
B = 1./variances
ratios = C/B
offset = np.sum(C)*np.sum(C)/(2*np.sum(B))

forsorting = ratios.argsort()

sorted_means = means[forsorting[::-1]]
sorted_stddevs = stddevs[forsorting[::-1]]
sorted_variances = variances[forsorting[::-1]]
sorted_vals = vals[forsorting[::-1]]
sorted_C = C[forsorting[::-1]]
sorted_B = B[forsorting[::-1]]
sorted_ratios = ratios[forsorting[::-1]]

means = sorted_means
stddevs = sorted_stddevs
variances = sorted_variances
vals = sorted_vals
C = sorted_C
B = sorted_B
ratios = sorted_ratios

In [3]:
maxes = {}
argmaxes = {}
Tmax = 100

# maxes[(n,T)] is the maximum score for dividing elements n..(N-1) into T partitions
# argmaxes[(n,T)] is the start of the second partition, i.e., first partition goes from n..(argmaxes[(n,T)]-1)

# base case.  maxes[(n,1)] = (\sum_{i=n..N-1} C_i)^2/(\sum_{i-n..N-1} B_i)
cumulative_C = 0.
cumulative_B = 0.
for n in np.arange(N-1,-1,-1):
    cumulative_C += C[n]
    cumulative_B += B[n]
    maxes[(n,1)] = cumulative_C*cumulative_C / cumulative_B
    argmaxes[(n,1)] = N
    
# increment case.  maxes[(n,T)] = max_{k=n+1..N-1} (\sum_{i=n..k-1} C_i)^2/(\sum_{i-n..k-1} B_k) + maxes[(k,T-1)] 
for T in np.arange(2,Tmax+1,1):
    for n in range(N):
        bestscore = -1E10
        bestk = -1
        cumulative_C = 0.
        cumulative_B = 0.
        for k in np.arange(n+1,N):
            cumulative_C += C[k-1]
            cumulative_B += B[k-1]
            tempscore = (cumulative_C*cumulative_C / cumulative_B) + maxes[(k,T-1)]
            if (tempscore > bestscore):
                bestscore = tempscore
                bestk = k
        maxes[(n,T)] = bestscore
        argmaxes[(n,T)] = bestk

In [7]:
# best partitions

for T in np.arange(1,Tmax+1,1):
    print ("Best partition of size",T,"with score",maxes[(0,T)]/2. - offset,":")
    current = 0
    for t in np.arange(T,0,-1):
        thenext = argmaxes[(current,t)]
        print ("(",current,",",thenext-1,")")
        current = thenext
        
for T in np.arange(1,Tmax+1,1):
    print(maxes[(0,T)]/2. - offset)

Best partition of size 1 with score -3.055333763768431e-13 :
( 0 , 999 )
Best partition of size 2 with score 2173.777040114511 :
( 0 , 519 )
( 520 , 999 )
Best partition of size 3 with score 2571.0909464216875 :
( 0 , 326 )
( 327 , 666 )
( 667 , 999 )
Best partition of size 4 with score 2729.9412484183 :
( 0 , 234 )
( 235 , 535 )
( 536 , 790 )
( 791 , 999 )
Best partition of size 5 with score 2815.3854447912745 :
( 0 , 153 )
( 154 , 369 )
( 370 , 602 )
( 603 , 828 )
( 829 , 999 )
Best partition of size 6 with score 2857.2579877823964 :
( 0 , 138 )
( 139 , 326 )
( 327 , 519 )
( 520 , 675 )
( 676 , 854 )
( 855 , 999 )
Best partition of size 7 with score 2882.824098643702 :
( 0 , 44 )
( 45 , 173 )
( 174 , 341 )
( 342 , 526 )
( 527 , 681 )
( 682 , 857 )
( 858 , 999 )
Best partition of size 8 with score 2907.1221543305874 :
( 0 , 44 )
( 45 , 173 )
( 174 , 335 )
( 336 , 498 )
( 499 , 627 )
( 628 , 784 )
( 785 , 914 )
( 915 , 999 )
Best partition of size 9 with score 2920.303833597921 :
( 0 ,

( 6 , 21 )
( 22 , 42 )
( 43 , 69 )
( 70 , 92 )
( 93 , 122 )
( 123 , 152 )
( 153 , 191 )
( 192 , 234 )
( 235 , 274 )
( 275 , 314 )
( 315 , 341 )
( 342 , 373 )
( 374 , 423 )
( 424 , 467 )
( 468 , 511 )
( 512 , 537 )
( 538 , 573 )
( 574 , 602 )
( 603 , 633 )
( 634 , 685 )
( 686 , 732 )
( 733 , 772 )
( 773 , 810 )
( 811 , 844 )
( 845 , 879 )
( 880 , 910 )
( 911 , 941 )
( 942 , 965 )
( 966 , 980 )
( 981 , 991 )
( 992 , 998 )
( 999 , 999 )
Best partition of size 35 with score 2983.120885143378 :
( 0 , 1 )
( 2 , 5 )
( 6 , 21 )
( 22 , 42 )
( 43 , 69 )
( 70 , 92 )
( 93 , 122 )
( 123 , 152 )
( 153 , 191 )
( 192 , 234 )
( 235 , 274 )
( 275 , 314 )
( 315 , 341 )
( 342 , 373 )
( 374 , 423 )
( 424 , 467 )
( 468 , 511 )
( 512 , 537 )
( 538 , 573 )
( 574 , 602 )
( 603 , 633 )
( 634 , 675 )
( 676 , 711 )
( 712 , 750 )
( 751 , 784 )
( 785 , 822 )
( 823 , 854 )
( 855 , 887 )
( 888 , 914 )
( 915 , 944 )
( 945 , 967 )
( 968 , 981 )
( 982 , 991 )
( 992 , 998 )
( 999 , 999 )
Best partition of size 36 with sc

( 0 , 1 )
( 2 , 5 )
( 6 , 13 )
( 14 , 26 )
( 27 , 42 )
( 43 , 66 )
( 67 , 81 )
( 82 , 96 )
( 97 , 117 )
( 118 , 138 )
( 139 , 166 )
( 167 , 189 )
( 190 , 209 )
( 210 , 234 )
( 235 , 256 )
( 257 , 276 )
( 277 , 314 )
( 315 , 339 )
( 340 , 369 )
( 370 , 396 )
( 397 , 431 )
( 432 , 465 )
( 466 , 488 )
( 489 , 514 )
( 515 , 538 )
( 539 , 573 )
( 574 , 598 )
( 599 , 624 )
( 625 , 647 )
( 648 , 672 )
( 673 , 693 )
( 694 , 721 )
( 722 , 750 )
( 751 , 771 )
( 772 , 784 )
( 785 , 810 )
( 811 , 833 )
( 834 , 854 )
( 855 , 881 )
( 882 , 904 )
( 905 , 926 )
( 927 , 941 )
( 942 , 956 )
( 957 , 971 )
( 972 , 984 )
( 985 , 991 )
( 992 , 996 )
( 997 , 998 )
( 999 , 999 )
Best partition of size 50 with score 2985.46186780781 :
( 0 , 1 )
( 2 , 5 )
( 6 , 11 )
( 12 , 15 )
( 16 , 26 )
( 27 , 42 )
( 43 , 66 )
( 67 , 81 )
( 82 , 96 )
( 97 , 117 )
( 118 , 138 )
( 139 , 166 )
( 167 , 189 )
( 190 , 209 )
( 210 , 234 )
( 235 , 256 )
( 257 , 276 )
( 277 , 314 )
( 315 , 339 )
( 340 , 369 )
( 370 , 396 )
( 397 , 43

( 622 , 633 )
( 634 , 655 )
( 656 , 672 )
( 673 , 688 )
( 689 , 711 )
( 712 , 732 )
( 733 , 750 )
( 751 , 771 )
( 772 , 784 )
( 785 , 810 )
( 811 , 833 )
( 834 , 854 )
( 855 , 874 )
( 875 , 889 )
( 890 , 904 )
( 905 , 920 )
( 921 , 934 )
( 935 , 945 )
( 946 , 956 )
( 957 , 971 )
( 972 , 978 )
( 979 , 984 )
( 985 , 991 )
( 992 , 996 )
( 997 , 998 )
( 999 , 999 )
Best partition of size 61 with score 2986.2024612798227 :
( 0 , 1 )
( 2 , 3 )
( 4 , 5 )
( 6 , 11 )
( 12 , 15 )
( 16 , 26 )
( 27 , 37 )
( 38 , 45 )
( 46 , 66 )
( 67 , 81 )
( 82 , 96 )
( 97 , 117 )
( 118 , 134 )
( 135 , 152 )
( 153 , 173 )
( 174 , 191 )
( 192 , 209 )
( 210 , 234 )
( 235 , 256 )
( 257 , 276 )
( 277 , 314 )
( 315 , 336 )
( 337 , 355 )
( 356 , 370 )
( 371 , 391 )
( 392 , 419 )
( 420 , 440 )
( 441 , 467 )
( 468 , 488 )
( 489 , 511 )
( 512 , 535 )
( 536 , 554 )
( 555 , 575 )
( 576 , 598 )
( 599 , 621 )
( 622 , 633 )
( 634 , 655 )
( 656 , 672 )
( 673 , 688 )
( 689 , 711 )
( 712 , 732 )
( 733 , 750 )
( 751 , 771 )
( 772 

( 154 , 173 )
( 174 , 191 )
( 192 , 209 )
( 210 , 229 )
( 230 , 240 )
( 241 , 258 )
( 259 , 275 )
( 276 , 301 )
( 302 , 317 )
( 318 , 336 )
( 337 , 355 )
( 356 , 370 )
( 371 , 391 )
( 392 , 419 )
( 420 , 440 )
( 441 , 467 )
( 468 , 488 )
( 489 , 511 )
( 512 , 525 )
( 526 , 537 )
( 538 , 554 )
( 555 , 573 )
( 574 , 590 )
( 591 , 602 )
( 603 , 623 )
( 624 , 634 )
( 635 , 655 )
( 656 , 672 )
( 673 , 688 )
( 689 , 711 )
( 712 , 732 )
( 733 , 750 )
( 751 , 771 )
( 772 , 784 )
( 785 , 810 )
( 811 , 828 )
( 829 , 844 )
( 845 , 857 )
( 858 , 874 )
( 875 , 889 )
( 890 , 904 )
( 905 , 920 )
( 921 , 933 )
( 934 , 941 )
( 942 , 952 )
( 953 , 961 )
( 962 , 971 )
( 972 , 978 )
( 979 , 984 )
( 985 , 989 )
( 990 , 991 )
( 992 , 994 )
( 995 , 997 )
( 998 , 998 )
( 999 , 999 )
Best partition of size 71 with score 2986.605043944336 :
( 0 , 1 )
( 2 , 3 )
( 4 , 5 )
( 6 , 11 )
( 12 , 15 )
( 16 , 26 )
( 27 , 37 )
( 38 , 45 )
( 46 , 66 )
( 67 , 81 )
( 82 , 96 )
( 97 , 114 )
( 115 , 128 )
( 129 , 138 )
( 139 ,

( 733 , 750 )
( 751 , 771 )
( 772 , 784 )
( 785 , 810 )
( 811 , 828 )
( 829 , 844 )
( 845 , 854 )
( 855 , 862 )
( 863 , 874 )
( 875 , 887 )
( 888 , 895 )
( 896 , 904 )
( 905 , 920 )
( 921 , 933 )
( 934 , 941 )
( 942 , 952 )
( 953 , 961 )
( 962 , 971 )
( 972 , 978 )
( 979 , 984 )
( 985 , 989 )
( 990 , 991 )
( 992 , 994 )
( 995 , 997 )
( 998 , 998 )
( 999 , 999 )
Best partition of size 79 with score 2986.778858430806 :
( 0 , 1 )
( 2 , 3 )
( 4 , 5 )
( 6 , 11 )
( 12 , 15 )
( 16 , 26 )
( 27 , 35 )
( 36 , 42 )
( 43 , 47 )
( 48 , 66 )
( 67 , 79 )
( 80 , 86 )
( 87 , 96 )
( 97 , 114 )
( 115 , 128 )
( 129 , 138 )
( 139 , 153 )
( 154 , 173 )
( 174 , 189 )
( 190 , 203 )
( 204 , 213 )
( 214 , 229 )
( 230 , 240 )
( 241 , 258 )
( 259 , 275 )
( 276 , 301 )
( 302 , 314 )
( 315 , 326 )
( 327 , 339 )
( 340 , 355 )
( 356 , 370 )
( 371 , 390 )
( 391 , 412 )
( 413 , 431 )
( 432 , 444 )
( 445 , 465 )
( 466 , 476 )
( 477 , 491 )
( 492 , 511 )
( 512 , 525 )
( 526 , 537 )
( 538 , 554 )
( 555 , 573 )
( 574 , 590

( 204 , 213 )
( 214 , 229 )
( 230 , 240 )
( 241 , 258 )
( 259 , 275 )
( 276 , 301 )
( 302 , 314 )
( 315 , 326 )
( 327 , 339 )
( 340 , 355 )
( 356 , 370 )
( 371 , 390 )
( 391 , 412 )
( 413 , 431 )
( 432 , 444 )
( 445 , 465 )
( 466 , 476 )
( 477 , 491 )
( 492 , 511 )
( 512 , 525 )
( 526 , 537 )
( 538 , 554 )
( 555 , 573 )
( 574 , 590 )
( 591 , 601 )
( 602 , 620 )
( 621 , 625 )
( 626 , 634 )
( 635 , 655 )
( 656 , 672 )
( 673 , 685 )
( 686 , 696 )
( 697 , 711 )
( 712 , 731 )
( 732 , 746 )
( 747 , 757 )
( 758 , 772 )
( 773 , 783 )
( 784 , 796 )
( 797 , 810 )
( 811 , 828 )
( 829 , 844 )
( 845 , 854 )
( 855 , 862 )
( 863 , 874 )
( 875 , 887 )
( 888 , 895 )
( 896 , 904 )
( 905 , 914 )
( 915 , 924 )
( 925 , 934 )
( 935 , 941 )
( 942 , 952 )
( 953 , 961 )
( 962 , 971 )
( 972 , 978 )
( 979 , 984 )
( 985 , 989 )
( 990 , 991 )
( 992 , 994 )
( 995 , 997 )
( 998 , 998 )
( 999 , 999 )
Best partition of size 87 with score 2986.9031610109146 :
( 0 , 1 )
( 2 , 3 )
( 4 , 5 )
( 6 , 7 )
( 8 , 11 )
( 12 , 15

( 190 , 203 )
( 204 , 213 )
( 214 , 229 )
( 230 , 240 )
( 241 , 258 )
( 259 , 274 )
( 275 , 283 )
( 284 , 303 )
( 304 , 315 )
( 316 , 326 )
( 327 , 339 )
( 340 , 355 )
( 356 , 370 )
( 371 , 390 )
( 391 , 412 )
( 413 , 431 )
( 432 , 444 )
( 445 , 465 )
( 466 , 476 )
( 477 , 491 )
( 492 , 511 )
( 512 , 525 )
( 526 , 537 )
( 538 , 554 )
( 555 , 573 )
( 574 , 590 )
( 591 , 601 )
( 602 , 620 )
( 621 , 625 )
( 626 , 634 )
( 635 , 655 )
( 656 , 672 )
( 673 , 685 )
( 686 , 696 )
( 697 , 711 )
( 712 , 722 )
( 723 , 736 )
( 737 , 750 )
( 751 , 757 )
( 758 , 772 )
( 773 , 783 )
( 784 , 796 )
( 797 , 810 )
( 811 , 826 )
( 827 , 835 )
( 836 , 844 )
( 845 , 854 )
( 855 , 862 )
( 863 , 874 )
( 875 , 887 )
( 888 , 895 )
( 896 , 904 )
( 905 , 914 )
( 915 , 924 )
( 925 , 934 )
( 935 , 941 )
( 942 , 950 )
( 951 , 956 )
( 957 , 965 )
( 966 , 971 )
( 972 , 975 )
( 976 , 980 )
( 981 , 984 )
( 985 , 989 )
( 990 , 991 )
( 992 , 994 )
( 995 , 997 )
( 998 , 998 )
( 999 , 999 )
Best partition of size 94 with sco

( 855 , 862 )
( 863 , 874 )
( 875 , 887 )
( 888 , 895 )
( 896 , 904 )
( 905 , 914 )
( 915 , 924 )
( 925 , 934 )
( 935 , 940 )
( 941 , 945 )
( 946 , 952 )
( 953 , 956 )
( 957 , 965 )
( 966 , 971 )
( 972 , 975 )
( 976 , 980 )
( 981 , 984 )
( 985 , 989 )
( 990 , 991 )
( 992 , 994 )
( 995 , 997 )
( 998 , 998 )
( 999 , 999 )
Best partition of size 100 with score 2987.0484793799956 :
( 0 , 1 )
( 2 , 3 )
( 4 , 5 )
( 6 , 7 )
( 8 , 11 )
( 12 , 15 )
( 16 , 21 )
( 22 , 26 )
( 27 , 35 )
( 36 , 42 )
( 43 , 46 )
( 47 , 61 )
( 62 , 69 )
( 70 , 79 )
( 80 , 86 )
( 87 , 96 )
( 97 , 105 )
( 106 , 117 )
( 118 , 128 )
( 129 , 138 )
( 139 , 151 )
( 152 , 166 )
( 167 , 173 )
( 174 , 189 )
( 190 , 203 )
( 204 , 213 )
( 214 , 229 )
( 230 , 240 )
( 241 , 258 )
( 259 , 274 )
( 275 , 283 )
( 284 , 303 )
( 304 , 315 )
( 316 , 326 )
( 327 , 339 )
( 340 , 355 )
( 356 , 369 )
( 370 , 383 )
( 384 , 396 )
( 397 , 416 )
( 417 , 431 )
( 432 , 441 )
( 442 , 456 )
( 457 , 467 )
( 468 , 476 )
( 477 , 488 )
( 489 , 498 )
( 4

In [3]:
vals

array([ 1.05162619e+01,  7.23485883e+00,  8.90083012e+00,  7.43308034e+00,
        7.90082310e+00,  5.99901129e+00,  6.82497650e+00,  8.25676803e+00,
        7.24981823e+00,  7.47174782e+00,  5.93872054e+00,  6.98525803e+00,
        7.26522839e+00,  7.35137078e+00,  4.81154591e+00,  6.33385544e+00,
        7.94610475e+00,  5.96813307e+00,  7.12844353e+00,  6.66205381e+00,
        5.87504940e+00,  4.82365508e+00,  6.55656877e+00,  6.08403890e+00,
        7.22713924e+00,  6.38966246e+00,  7.41112644e+00,  7.20815174e+00,
        6.72487173e+00,  6.67463680e+00,  7.29508314e+00,  5.61268874e+00,
        6.94219759e+00,  5.80524391e+00,  7.41899021e+00,  7.22534674e+00,
        5.77564273e+00,  5.26524811e+00,  6.80007954e+00,  5.31848347e+00,
        3.01432072e+00,  5.61119699e+00,  7.22988644e+00,  6.44462513e+00,
        5.10376486e+00,  6.57092840e+00,  3.69820293e+00,  3.20235268e+00,
        5.70992412e+00,  5.82351270e+00,  5.30419687e+00,  5.71624854e+00,
        6.00118297e+00,  

In [2]:
sorted_ratios

array([ 1.04209884e+01,  1.01867357e+01,  8.71388988e+00,  8.71154676e+00,
        8.05946670e+00,  7.94670456e+00,  7.39544114e+00,  7.19724975e+00,
        7.12164534e+00,  7.07219617e+00,  7.06811210e+00,  6.98070434e+00,
        6.80249632e+00,  6.73052815e+00,  6.62736387e+00,  6.61642152e+00,
        6.47367979e+00,  6.41735713e+00,  6.40673960e+00,  6.40542636e+00,
        6.38736626e+00,  6.38710233e+00,  6.33333396e+00,  6.29547552e+00,
        6.25960046e+00,  6.24119061e+00,  6.22134405e+00,  6.10983048e+00,
        6.05450296e+00,  6.03472197e+00,  6.02310967e+00,  6.00290697e+00,
        5.99536298e+00,  5.98786936e+00,  5.97041208e+00,  5.96278109e+00,
        5.87568887e+00,  5.87217343e+00,  5.81016097e+00,  5.79978463e+00,
        5.78362209e+00,  5.70947167e+00,  5.67828205e+00,  5.58458810e+00,
        5.54331872e+00,  5.50802781e+00,  5.44905798e+00,  5.40714258e+00,
        5.39011729e+00,  5.35087260e+00,  5.34863204e+00,  5.33918800e+00,
        5.32046737e+00,  

In [5]:
cumulative_C

NameError: name 'cumulative_C' is not defined